I will start with a standard random forest (RF) as base model. Then we will tune hyperparams of RF to achieve a better model. Finally, I will try XGBoost.

We will pick features based on analysis from EDA notebook.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import  mean_absolute_error, mean_squared_error, mean_squared_log_error

In [48]:
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [3]:
# folder = '/kaggle/input/house-prices-advanced-regression-techniques/'
folder = 'data'

train = pd.read_csv(os.path.join(folder, 'train.csv'))
test = pd.read_csv(os.path.join(folder, 'test.csv'))

# exclude outliers
# train = train.query('GrLivArea <= 4000' )
# test = test.query('GrLivArea <= 4000')
print(train.shape)
print(test.shape)

(1460, 81)
(1459, 80)


In [4]:
# concat train and test sets st we always perform transformation on both sets
test['SalePrice'] = 0
data = pd.concat([train, test])

In [5]:
# lowercase all column names for convenience
data.columns = [str.lower(cc) for cc in data.columns]

# sale price in thousands is better for plotting
data['sale_price_in_thousand'] = data['saleprice']/(10**3)

In [6]:
# sale price per square feet is also interested
data['sale_price_per_sf'] = data['saleprice'] / data['grlivarea']

There are two approaches:
+ predict directly sale price
+ predict price per SF, then multiply with living area to estimate sale price

I will try both, but first we need some helpers.

### Helper methods

In [7]:
def cal_age_from_built(row):
    return row['yrsold'] - row['yearbuilt']

def cal_age_from_remodel(row):
    return row['yrsold'] - row['yearremodadd']


def fold_zone_type(ms_zone):
    if ms_zone in ['FV', 'RH', 'C (all)']:
        return 'Other'
    else:
        return ms_zone
#         return {'RL': 'Residential Low Density'.lower() , 
#                 'RM': 'Residential Medium Density'.lower(),
#                 None: 'NA'
#                }[ms_zone]    

def to_adjacency(cond):
    if 'RR' in cond:
        return 'Railroad'
    if 'Pos' in cond:
        return 'Positive feature'
    return {
        'Artery': 'Arterial street',
        'Feedr': 'Feeder street',
        'Norm': 'Normal'    
        }[cond]

In [8]:
def to_quantitative(text_feat, df, scoring):
    '''
    Given a feature stored in data as text but actually a quantitative feat, convert it to numerical values
    via given encoding
    :param scoring:
    :param text_feat:
    :return:
    '''
    n_na = sum(df[text_feat].isnull())
    print('\t Feature {0} has {1} NAs, they will be filled by 0'.format(text_feat, n_na))

    res = df.copy()
    res[text_feat].fillna("NA", inplace=True)

    # print('\t Column {} has {} NAs, they will be filled by forward filling'.format(text_feat, n_na))
    # res[text_feat].fillna(method='ffill', inplace=True)
    res[text_feat] = res[text_feat].apply(lambda form: scoring[form])
    return res

def quant_to_scores(quant_feats, data, scorings):
    print('\n Converting quantitative text features to scores...')
    score_dict = dict(zip(quant_feats, scorings))
    
    for tf in quant_feats:  
        data = to_quantitative(text_feat=tf, df=data, scoring=score_dict[tf])

    return data

In [9]:
def onehot_encode(cat_feat, data, dummy_na=False):
    encoded = pd.get_dummies(data[cat_feat], prefix=cat_feat, dummy_na=dummy_na)
    res = pd.concat([data.drop(columns=[cat_feat]), encoded], axis='columns')
    return res

def encode_cat_feats(data, cat_feats, dummy_na=False):
    print('Onehot encode categorical features: ', cat_feats)

    encoded_df = data.copy()
    # encode 1 cat feature at a time
    for cf in cat_feats:
        encoded_df = onehot_encode(cf, encoded_df, dummy_na=dummy_na)

    return encoded_df

In [10]:
def list_string_columns(data):
    return list(data.columns[np.where(data.dtypes == 'object')])

In [11]:
def split_train_valid(data, target):
    y = data.pop(target)
    X = data
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=.1, 
                                                          random_state=1
                                                         )
    return X_train, X_valid, y_train, y_valid

In [12]:
def check_na(data):
    # check if any NA left
    na_count = [sum(data[ff].isnull()) for ff in data.columns]
    res = pd.DataFrame({'column': data.columns, 'na_count': na_count})
    res['na_fraction'] = res['na_count']/len(data)
    return res.query('na_count > 0')

In [13]:
def list_numeric_columns(data):
    return list(data.columns[np.where(data.dtypes != 'object')])

In [14]:
def make_output(y_pred):
    test_index = range(len(train) + 1, len(data) + 1)
    return pd.DataFrame({'Id': test_index, 'SalePrice': y_pred})

In [15]:
# for converting quantitative text feats to scores
six_scale = {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0}
quant_feats = ['utilities',
 'exterqual',
 'extercond',
 'heatingqc',
 'bsmtqual',
 'bsmtcond',
 'kitchenqual',
 'bsmtexposure',
 'bsmtfintype1'
]
scorings = [{"AllPub": 4, "NoSewr": 3, "NoSeWa": 2, "ELO": 1, "NA": 0},
            six_scale,
            six_scale,
            six_scale,
            six_scale,
            six_scale,
            six_scale,
            {"Gd": 4, "Av": 3, "Mn": 2, "No": 1, "NA": 0},
            {"GLQ": 6, "ALQ": 5, "BLQ": 4, "Rec": 3, "LwQ": 2, "Unf": 1, "NA": 0},
            ]
# len(quant_feats) == len(scorings)

## Preprocessing

In [27]:
target = 'saleprice'

In [16]:
na_checker = check_na(data)
na_checker = na_checker.sort_values('na_count', ascending=False)
na_checker

,column,na_count,na_fraction
72,poolqc,2909,0.996574
74,miscfeature,2814,0.964029
6,alley,2721,0.932169
73,fence,2348,0.804385
57,fireplacequ,1420,0.486468
3,lotfrontage,486,0.166495
60,garagefinish,159,0.054471
63,garagequal,159,0.054471
64,garagecond,159,0.054471
59,garageyrblt,159,0.054471


Drop columns with lots of NAs.

In [17]:
columns_with_lots_of_na = na_checker[na_checker['na_fraction'] > .3] ['column']
print(columns_with_lots_of_na)
data = data.drop(columns=columns_with_lots_of_na)

72         poolqc
74    miscfeature
6           alley
73          fence
57    fireplacequ
Name: column, dtype: object


### For numeric columns, fill NAs by mean

In [18]:
num_vars = list_numeric_columns(data)
data[num_vars].mean()
data[num_vars] = data[num_vars].fillna(data[num_vars].mean())

### Convert quantitative text columns to scores

In [19]:
data = quant_to_scores(quant_feats, data, scorings)


 Converting quantitative text features to scores...
	 Feature utilities has 2 NAs, they will be filled by 0
	 Feature exterqual has 0 NAs, they will be filled by 0
	 Feature extercond has 0 NAs, they will be filled by 0
	 Feature heatingqc has 0 NAs, they will be filled by 0
	 Feature bsmtqual has 81 NAs, they will be filled by 0
	 Feature bsmtcond has 82 NAs, they will be filled by 0
	 Feature kitchenqual has 1 NAs, they will be filled by 0
	 Feature bsmtexposure has 82 NAs, they will be filled by 0
	 Feature bsmtfintype1 has 79 NAs, they will be filled by 0


In [20]:
data[quant_feats].describe()

,utilities,exterqual,extercond,heatingqc,bsmtqual,bsmtcond,kitchenqual,bsmtexposure,bsmtfintype1
count,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000
mean,3.996574,3.396711,3.085646,4.151764,3.477561,2.918465,3.509764,1.623844,3.541624
std,0.111020,0.580293,0.372361,0.957952,0.905448,0.574950,0.665273,1.070026,2.113851
min,0.000000,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,1.000000,1.000000
50%,4.000000,3.000000,3.000000,5.000000,4.000000,3.000000,3.000000,1.000000,4.000000
75%,4.000000,4.000000,3.000000,5.000000,4.000000,3.000000,4.000000,2.000000,6.000000
max,4.000000,5.000000,5.000000,5.000000,5.000000,4.000000,5.000000,4.000000,6.000000


### Compute derived features

+ age from built
+ aggr features by neighborhood:
    + mean price
    + diff from avg house size
    + diff from avg lot size
    + diff from avg age
    + diff from avg house quality

In [21]:
data['age_from_built'] = data.apply(axis=1, func=cal_age_from_built)

In [22]:
data['adjacency'] = data['condition1'].apply(to_adjacency)

In [23]:
# TODO: drop non-important feats
to_drop = [ 'garagetype', 'garagefinish', 'garagequal', 'garagecond',
            'condition1', 'condition2', 
           'bsmtfintype2', 'street',
          'landslope', 'landcontour', 'bldgtype', 
           'heating',
            'roofmatl', 
           'masvnrtype',
          ]
#            'centralair','electrical', 
#            'functional', 'paveddrive',
#            'saletype', 'salecondition',

In [24]:
data = data.drop(columns=to_drop)

In [25]:
to_aggreg = ['saleprice', 'lotarea', 'grlivarea']
avg_by_nbh = data.groupby('neighborhood')[to_aggreg].mean().add_prefix('avg_').reset_index()
avg_by_nbh

,neighborhood,avg_saleprice,avg_lotarea,avg_grlivarea
0,Blmngtn,118314.464286,3399.357143,1404.892857
1,Blueste,27500.000000,2346.500000,1159.700000
2,BrDale,55730.000000,1840.633333,1115.233333
3,BrkSide,67040.509259,6959.777778,1234.907407
4,ClearCr,135268.909091,24842.250000,1744.386364
5,CollgCr,111216.726592,9999.752809,1496.119850
6,Crawfor,104289.912621,11650.106796,1722.796117
7,Edwards,66092.628866,10355.536082,1337.737113
8,Gilbert,92336.400000,11342.369697,1620.896970
9,IDOTRR,39834.193548,8826.473118,1205.247312


In [ ]:
data = pd.merge(data, avg_by_nbh)

In [32]:
data['diff_from_avg_house_size_in_neighborhood'] = data['grlivarea'] - data['avg_grlivarea']
data['diff_from_avg_lot_size_in_neighborhood'] = data['lotarea'] - data['avg_lotarea']
data = data.rename(columns= {'avg_saleprice': 'avg_saleprice_in_neighborhood'})

In [64]:
data['house_land_ratio'] = data['grlivarea'] / data['lotarea']

In [65]:
train_part = data[data[target] > 0]
test_part = data[data[target]==0]
y_train = train_part[target]

In [ ]:
# list(data)

### Simple model
Linear regression with some more features.

In [66]:
base_feats = ['mosold', 'yrsold', 'lotfrontage', 'grlivarea', 'bedroomabvgr', 'house_land_ratio']
X_train = train_part[base_feats]
X_test = test_part[base_feats]

In [70]:
reg = RidgeCV(alphas=[.1, 1, 10], cv=5)
reg.fit(X_train, y_train)
ridge_res = make_output(reg.predict(X_test))
print(ridge_res['SalePrice'].head())
ridge_res.to_csv('ridge_pred.csv', index=False)

0    165093.350481
1    142413.338298
2    198069.068596
3    115702.050562
4     97833.416876
Name: SalePrice, dtype: float64


In [69]:
reg = LassoCV( cv=5)
reg.fit(X_train, y_train)
lasso_res = make_output(reg.predict(X_test))
lasso_res.SalePrice.head()

0    149675.230512
1    122311.665139
2    199717.548510
3    128806.519177
4    112835.566624
Name: SalePrice, dtype: float64

In [58]:
lasso_res.to_csv('lasso_pred.csv', index=False)

In [34]:
simple_feats = ['mosold', 'yrsold', 'lotarea', 'grlivarea', 'bedroomabvgr',
                'avg_saleprice_in_neighborhood', 
                'diff_from_avg_house_size_in_neighborhood',
                'diff_from_avg_lot_size_in_neighborhood'
               ]

X_train = train_part[simple_feats]
X_test = test_part[simple_feats]

In [36]:
# for linear reg
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

reg = LinearRegression()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

lin_res = make_output(y_pred)
lin_res.to_csv('lin_pred.csv', index=False)

In [37]:
print(lin_res.SalePrice.describe())

count      1459.000000
mean     180921.195890
std       66750.481038
min       50324.140672
25%      130092.720341
50%      170348.365089
75%      217925.072868
max      544280.510733
Name: SalePrice, dtype: float64


### Simple RF

In [38]:
rf = RandomForestRegressor(n_estimators=100, max_features=1.0, n_jobs=-1,
                               random_state=1,
                               )
rf.fit(X_train, y_train)

RandomForestRegressor(max_features=1.0, n_jobs=-1, random_state=1)

In [39]:
rf_res = make_output(rf.predict(X_test))
rf_res.to_csv('rf_pred0.csv', index=False)

In [40]:
rf_res.SalePrice.describe()

count      1459.000000
mean     185313.241892
std       75467.504985
min       64684.440000
25%      137135.625000
50%      164547.000000
75%      218696.970000
max      618050.760000
Name: SalePrice, dtype: float64

In [41]:
rf_res.head()

,Id,SalePrice
0,1461,191370.46
1,1462,142251.50
2,1463,207162.93
3,1464,149170.00
4,1465,136612.00


### Encode cat features

In [ ]:
cat_feats = list_string_columns(data)
print('# cat feats: ', len(cat_feats))
print(cat_feats)

In [ ]:
data[cat_feats].describe()

In [ ]:
dump_data = encode_cat_feats(data, cat_feats=cat_feats, dummy_na=True)
print(dump_data.shape)
# dump_data.head()

## Base RF

No tuning, no new features.

In [ ]:
encoded_train = dump_data.loc[dump_data[target] != 0].copy()
na_checker = check_na(encoded_train)
if not na_checker.empty:
    print(na_checker)

X_train, X_valid, y_train, y_valid = split_train_valid(encoded_train, target)

base_rf = RandomForestRegressor(n_estimators=100, max_features=1.0, n_jobs=-1,
                               random_state=1,
                               )
base_rf.fit(X_train, y_train)

y_pred = base_rf.predict(X_valid)
base_rmse = np.sqrt(mean_squared_error(y_valid, y_pred)) 
print('If predict directly sale price, base RMSE: ', round(base_rmse, 2))
base_rmsle = np.sqrt(mean_squared_log_error(y_valid, y_pred))
print('base RMSLE: ', base_rmsle)

In [ ]:
data[data.saleprice != 0].saleprice.describe()

As sale prices are at least 30K, this RMSE of less than 2K is really nice.
But we need to see if we have overfitting here. This will be in next version.

## Predict on test set

In [ ]:
X_test = dump_data.loc[dump_data[target] == 0].drop(columns=[target])
print(X_test.shape)

y_pred = base_rf.predict(X_test)

In [ ]:
res = make_output(y_pred)
res.SalePrice.describe()

In [ ]:
res.to_csv('pred1.csv', index=False)

## Run CV to avoid overfitting

Tune RF via randomized search.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 500, num = 5)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(4, 24, num = 6)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

In [ ]:
# base model
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, 
                               param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2, 
                               random_state=42, 
                               n_jobs = -1
                              )

In [ ]:
encoded_train = dump_data[dump_data[target] != 0].copy()
y = encoded_train.pop(target)
X = encoded_train

In [ ]:
rf_random.fit(X, y)

In [ ]:
rf_random.best_params_

In [ ]:
def cal_rmsle(estimator, X_valid, y_valid):
    y_pred = estimator.predict(X_valid)
    return np.sqrt(mean_squared_log_error(y_valid, y_pred))

In [ ]:
rf_best = rf_random.best_estimator_
print(cal_rmsle(rf_best, X_valid, y_valid))

In [ ]:
y_pred = rf_best.predict(X_test)
res = make_output(y_pred)
res.SalePrice.describe()

In [ ]:
res.to_csv('tuned_pred.csv', index=False)